In [ ]:
pip install pandas

In [ ]:
# ?Step 2: Import libraries
import pandas as pd
import numpy as np
from sdv.single_table import CTGANSynthesizer, TVAESynthesizer, GaussianCopulaSynthesizer
import os

In [ ]:
#
 
# Step 3: Define CSV file names
lookup_files = {
    'PolicyStatus': 'PolicyStatus.csv',
    'PolicyType': 'PolicyType.csv',
    'CoverageType': 'CoverageType.csv'
}
 
main_files = {
    'Customer': 'Customer.csv',
    'Policy': 'Policy.csv',
    'Coverage': 'Coverage.csv',
    'Claim': 'Claim.csv',
    'Beneficiary': 'Beneficiary.csv'
}
# Step 4: Load datasets
dataframes = {}
for name, file in {**lookup_files, **main_files}.items():
    dataframes[name] = pd.read_csv(file)
 
# Step 5: Synthesizer setup
def generate_synthetic(df, table_name, sample_size, model_type='ctgan'):
    if model_type == 'ctgan':
        model = CTGANSynthesizer()
    elif model_type == 'tvae':
        model = TVAESynthesizer()
    elif model_type == 'copula':
        model = GaussianCopulaSynthesizer()
    else:
        raise ValueError(f"Unknown model type: {model_type}")
    
    model.fit(df)  # Corrected line
    return model.sample(sample_size)

In [ ]:
# samples = {
#     'Customer': 1000,
#     'Policy': 1500,
#     'Coverage': 1500,
#     'Claim': 800,
#     'Beneficiary': 700
# }
 
# model_choices = {
#     'Customer': 'tvae',
#     'Policy': 'copula',
#     'Coverage': 'ctgan',
#     'Claim': 'ctgan',
#     'Beneficiary': 'tvae'
# }
 
# synthetic_data = {}
 
# # Generate Customer (independent)
# synthetic_data['Customer'] = generate_synthetic(dataframes['Customer'], 'Customer', samples['Customer'], model_choices['Customer'])
 
# # Add customerId if not generated
# if 'customerId' not in synthetic_data['Customer'].columns:
#     synthetic_data['Customer']['customerId'] = range(1, len(synthetic_data['Customer']) + 1)
 
# # Generate Policy
# policy_input = dataframes['Policy'].copy()
# policy_input['customerId'] = np.random.choice(synthetic_data['Customer']['customerId'], size=len(policy_input))
# policy_input['status_id'] = np.random.choice(dataframes['PolicyStatus']['status_id'], size=len(policy_input))
# policy_input['type_id'] = np.random.choice(dataframes['PolicyType']['type_id'], size=len(policy_input))
# synthetic_data['Policy'] = generate_synthetic(policy_input, 'Policy', samples['Policy'], model_choices['Policy'])
 
# # Add policyId if not present
# if 'policyId' not in synthetic_data['Policy'].columns:
#     synthetic_data['Policy']['policyId'] = range(1, len(synthetic_data['Policy']) + 1)
 
# # Generate Coverage
# coverage_input = dataframes['Coverage'].copy()
# coverage_input['policyId'] = np.random.choice(synthetic_data['Policy']['policyId'], size=len(coverage_input))
# coverage_input['coverage_type_id'] = np.random.choice(dataframes['CoverageType']['coverage_id'], size=len(coverage_input))
# synthetic_data['Coverage'] = generate_synthetic(coverage_input, 'Coverage', samples['Coverage'], model_choices['Coverage'])
 
# # Generate Claim
# claim_input = dataframes['Claim'].copy()
# claim_input['policyId'] = np.random.choice(synthetic_data['Policy']['policyId'], size=len(claim_input))
# synthetic_data['Claim'] = generate_synthetic(claim_input, 'Claim', samples['Claim'], model_choices['Claim'])
 
# # Generate Beneficiary
# beneficiary_input = dataframes['Beneficiary'].copy()
# beneficiary_input['policyId'] = np.random.choice(synthetic_data['Policy']['policyId'], size=len(beneficiary_input))
# synthetic_data['Beneficiary'] = generate_synthetic(beneficiary_input, 'Beneficiary', samples['Beneficiary'], model_choices['Beneficiary'])

In [ ]:
pip install sdv==1.10.0


In [ ]:
lookup_files = {
    'PolicyStatus': 'PolicyStatus.csv',
    'PolicyType': 'PolicyType.csv',
    'CoverageType': 'CoverageType.csv',
    'PaymentMethod': 'PaymentMethod.csv'
}

main_files = {
    'Customer': 'Customer.csv',
    'Policy': 'Policy.csv',
    'Coverage': 'Coverage.csv',
    'Premium': 'Premium.csv',
    'Claim': 'Claim.csv',
    'Beneficiary': 'Beneficiary.csv'
}


In [ ]:
import pandas as pd

data = {}
for name, file in {**lookup_files, **main_files}.items():
    data[name] = pd.read_csv(file)


In [ ]:
import sdv
print("SDV version:", sdv.__version__)

from sdv.metadata.multi_table import MultiTableMetadata
print("MultiTableMetadata add_table:", MultiTableMetadata.add_table)


In [ ]:
from sdv.metadata.multi_table import MultiTableMetadata

metadata = MultiTableMetadata()

# data is your dict of table_name: dataframe
metadata.detect_from_dataframes(data)




In [ ]:
print(help(metadata.add_table))
print(dir(metadata))


In [ ]:

from sdv.multi_table import HMASynthesizer

synthesizer = HMASynthesizer(metadata)
synthesizer.fit(data)

synthetic_data = synthesizer.sample()

# Save output
for table_name, df in synthetic_data.items():
    df.to_csv(f'SYN_{table_name}.csv', index=False)
    print(f'Saved SYN_{table_name}.csv')


In [ ]:
# # Relational GAN for Synthetic Data Generation in Jupyter Notebook
# # Author: Nidya & ChatGPT | Goal: Generate synthetic data for main tables using GAN, preserving referential integrity

# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from sklearn.preprocessing import LabelEncoder, MinMaxScaler
# from collections import defaultdict
# import os
# import random

# # Set seed for reproducibility
# torch.manual_seed(42)
# np.random.seed(42)
# random.seed(42)

# # ========== CONFIGURATION ==========
# main_tables_config = {
#     'Customer.csv': 300,
#     'Policy.csv': 500,
#     'Coverage.csv': 400,
#     'Premium.csv': 350,
#     'Claim.csv': 450,
#     'Beneficiary.csv': 300
# }
# lookup_tables = {
#     'PolicyType': pd.read_csv('PolicyType.csv'),
#     'PolicyStatus': pd.read_csv('PolicyStatus.csv'),
#     'CoverageType': pd.read_csv('CoverageType.csv'),
#     'PaymentMethod': pd.read_csv('PaymentMethod.csv')
# }

# # ========== UTILITY CLASSES ==========
# class SimpleGAN(nn.Module):
#     def __init__(self, input_dim, hidden_dim=128):
#         super().__init__()
#         self.generator = nn.Sequential(
#             nn.Linear(16, hidden_dim),
#             nn.ReLU(),
#             nn.Linear(hidden_dim, input_dim)
#         )
#         self.discriminator = nn.Sequential(
#             nn.Linear(input_dim, hidden_dim),
#             nn.LeakyReLU(0.2),
#             nn.Linear(hidden_dim, 1),
#             nn.Sigmoid()
#         )

#     def generate(self, z):
#         return self.generator(z)

#     def discriminate(self, x):
#         return self.discriminator(x)

# # ========== FUNCTION TO PROCESS AND TRAIN ==========
# def process_and_generate(table_path, table_name, synthetic_rows):
#     print(f"\nProcessing {table_name}...")
#     df = pd.read_csv(table_path).dropna().reset_index(drop=True)

#     encoders = {}
#     scalers = {}
#     encoded_df = pd.DataFrame()

#     # Encode categorical and scale numeric
#     for col in df.columns:
#         if df[col].dtype == 'object':
#             le = LabelEncoder()
#             encoded_df[col] = le.fit_transform(df[col])
#             encoders[col] = le
#         else:
#             scaler = MinMaxScaler()
#             encoded_df[col] = scaler.fit_transform(df[[col]])
#             scalers[col] = scaler

#     X = torch.tensor(encoded_df.values, dtype=torch.float32)

#     gan = SimpleGAN(input_dim=X.shape[1])
#     optimizer_G = optim.Adam(gan.generator.parameters(), lr=0.001)
#     optimizer_D = optim.Adam(gan.discriminator.parameters(), lr=0.001)
#     criterion = nn.BCELoss()

#     # Training loop
#     for epoch in range(500):
#         z = torch.randn(X.shape[0], 16)
#         fake = gan.generate(z)

#         real_labels = torch.ones(X.shape[0], 1)
#         fake_labels = torch.zeros(X.shape[0], 1)

#         # Train Discriminator
#         optimizer_D.zero_grad()
#         d_loss_real = criterion(gan.discriminate(X), real_labels)
#         d_loss_fake = criterion(gan.discriminate(fake.detach()), fake_labels)
#         d_loss = d_loss_real + d_loss_fake
#         d_loss.backward()
#         optimizer_D.step()

#         # Train Generator
#         optimizer_G.zero_grad()
#         g_loss = criterion(gan.discriminate(fake), real_labels)
#         g_loss.backward()
#         optimizer_G.step()

#         if epoch % 100 == 0:
#             print(f"Epoch {epoch}: D_loss={d_loss.item():.4f}, G_loss={g_loss.item():.4f}")

#     # Generate synthetic data
#     z = torch.randn(synthetic_rows, 16)
#     synthetic_data = gan.generate(z).detach().numpy()
#     synthetic_df = pd.DataFrame(synthetic_data, columns=df.columns)

#     # Inverse transform
#     for col in synthetic_df.columns:
#         if col in encoders:
#             synthetic_df[col] = synthetic_df[col].rank(pct=True)
#             synthetic_df[col] = (synthetic_df[col] * (len(encoders[col].classes_) - 1)).astype(int)
#             synthetic_df[col] = synthetic_df[col].clip(0, len(encoders[col].classes_) - 1)
#             synthetic_df[col] = encoders[col].inverse_transform(synthetic_df[col])
#         else:
#             synthetic_df[col] = scalers[col].inverse_transform(synthetic_df[[col]])

#     # Fix FK columns with lookup values
#     for lookup_name, lookup_df in lookup_tables.items():
#         for col in synthetic_df.columns:
#             if lookup_name.lower() in col.lower():
#                 values = lookup_df.iloc[:, 0].unique()
#                 synthetic_df[col] = np.random.choice(values, size=synthetic_df.shape[0])

#     output_path = f"synthetic_{table_name}.csv"
#     synthetic_df.to_csv(output_path, index=False)
#     print(f"Saved synthetic data: {output_path}")

# # ========== MAIN EXECUTION LOOP ==========
# for table_file, rows in main_tables_config.items():
#     table_name = table_file.replace('.csv', '')
#     process_and_generate(table_file, table_name, rows)

# print("\n✅ Synthetic data generation complete for all main tables.")


In [ ]:
# Relational GAN for Synthetic Data Generation in Jupyter Notebook
# Author: Nidya & ChatGPT | Goal: Generate synthetic data for main tables using GAN, preserving referential integrity

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from collections import defaultdict
import os
import random

# Set seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# ========== CONFIGURATION ==========
main_tables_config = {
    'Customer.csv': 300,
    'Policy.csv': 500,
    'Coverage.csv': 400,
    'Premium.csv': 350,
    'Claim.csv': 450,
    'Beneficiary.csv': 300
}
lookup_tables = {
    'PolicyType': pd.read_csv('PolicyType.csv'),
    'PolicyStatus': pd.read_csv('PolicyStatus.csv'),
    'CoverageType': pd.read_csv('CoverageType.csv'),
    'PaymentMethod': pd.read_csv('PaymentMethod.csv')
}
lookup_values = {key.lower(): df.iloc[:, 0].unique().tolist() for key, df in lookup_tables.items()}

# ========== UTILITY CLASSES ==========
class SimpleGAN(nn.Module):
    def __init__(self, input_dim, hidden_dim=128):
        super().__init__()
        self.generator = nn.Sequential(
            nn.Linear(16, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )
        self.discriminator = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )

    def generate(self, z):
        return self.generator(z)

    def discriminate(self, x):
        return self.discriminator(x)

# ========== FUNCTION TO PROCESS AND TRAIN ==========
def process_and_generate(table_path, table_name, synthetic_rows):
    print(f"\nProcessing {table_name}...")
    df = pd.read_csv(table_path).dropna().reset_index(drop=True)

    encoders = {}
    scalers = {}
    encoded_df = pd.DataFrame()

    for col in df.columns:
        if df[col].dtype == 'object':
            le = LabelEncoder()
            encoded_df[col] = le.fit_transform(df[col])
            encoders[col] = le
        else:
            scaler = MinMaxScaler()
            encoded_df[col] = scaler.fit_transform(df[[col]]).ravel()  # <== FIXED HERE
            scalers[col] = scaler


    X = torch.tensor(encoded_df.values, dtype=torch.float32)

    gan = SimpleGAN(input_dim=X.shape[1])
    optimizer_G = optim.Adam(gan.generator.parameters(), lr=0.001)
    optimizer_D = optim.Adam(gan.discriminator.parameters(), lr=0.001)
    criterion = nn.BCELoss()

    # Training loop
    for epoch in range(500):
        z = torch.randn(X.shape[0], 16)
        fake = gan.generate(z)

        real_labels = torch.ones(X.shape[0], 1)
        fake_labels = torch.zeros(X.shape[0], 1)

        # Train Discriminator
        optimizer_D.zero_grad()
        d_loss_real = criterion(gan.discriminate(X), real_labels)
        d_loss_fake = criterion(gan.discriminate(fake.detach()), fake_labels)
        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()
        optimizer_D.step()

        # Train Generator
        optimizer_G.zero_grad()
        g_loss = criterion(gan.discriminate(fake), real_labels)
        g_loss.backward()
        optimizer_G.step()

        if epoch % 100 == 0:
            print(f"Epoch {epoch}: D_loss={d_loss.item():.4f}, G_loss={g_loss.item():.4f}")

    # Generate synthetic data
    z = torch.randn(synthetic_rows, 16)
    synthetic_data = gan.generate(z).detach().numpy()
    synthetic_df = pd.DataFrame(synthetic_data, columns=df.columns)

    # Inverse transform
    for col in synthetic_df.columns:
        if col in encoders:
            synthetic_df[col] = synthetic_df[col].rank(pct=True)
            synthetic_df[col] = (synthetic_df[col] * (len(encoders[col].classes_) - 1)).astype(int)
            synthetic_df[col] = synthetic_df[col].clip(0, len(encoders[col].classes_) - 1)
            synthetic_df[col] = encoders[col].inverse_transform(synthetic_df[col])
        else:
            synthetic_df[col] = scalers[col].inverse_transform(synthetic_df[[col]])

    # Fix FK columns with lookup values (preserve referential integrity)
    for col in synthetic_df.columns:
        col_lower = col.lower()
        for lookup_key, valid_values in lookup_values.items():
            if lookup_key in col_lower:
                synthetic_df[col] = np.random.choice(valid_values, size=len(synthetic_df))

    output_path = f"synthetic_{table_name}.csv"
    synthetic_df.to_csv(output_path, index=False)
    print(f"Saved synthetic data: {output_path}")

# ========== MAIN EXECUTION LOOP ==========
for table_file, rows in main_tables_config.items():
    table_name = table_file.replace('.csv', '')
    process_and_generate(table_file, table_name, rows)

print("\n✅ Synthetic data generation complete for all main tables.")


In [ ]:
pip install --upgrade sdv


In [ ]:
import sys
print("Python executable:", sys.executable)


In [ ]:
import sys
!{sys.executable} -m pip uninstall -y sdv
!{sys.executable} -m pip install sdv



In [ ]:
!{sys.executable} -m pip show sdv


In [ ]:
# Install SDV if not installed
# !pip install sdv pandas numpy

import pandas as pd
import numpy as np
from sdv.tabular import CTGAN

In [ ]:
from sdv.tabular import CTGAN
print("CTGAN imported successfully!")


In [ ]:
pip install sdv --upgrade


In [ ]:
# ✅ Relational Synthetic Data Generation using SDV HMA1 (Hierarchical Multi-table Architecture)
# Author: Nidya & ChatGPT

from sdv.multi_table import HMASynthesizer
from sdv.metadata import MultiTableMetadata
import pandas as pd

# ========== LOAD DATA ==========
# Please ensure these CSV files are in the current directory
main_tables = {
    'Customer': pd.read_csv('Customer.csv'),
    'Policy': pd.read_csv('Policy.csv'),
    'Coverage': pd.read_csv('Coverage.csv'),
    'Premium': pd.read_csv('Premium.csv'),
    'Claim': pd.read_csv('Claim.csv'),
    'Beneficiary': pd.read_csv('Beneficiary.csv')
}

lookup_tables = {
    'PolicyStatus': pd.read_csv('PolicyStatus.csv'),
    'PolicyType': pd.read_csv('PolicyType.csv'),
    'CoverageType': pd.read_csv('CoverageType.csv'),
    'PaymentMethod': pd.read_csv('PaymentMethod.csv')
}

all_tables = {**main_tables, **lookup_tables}

In [ ]:


# ========== DEFINE METADATA ==========
metadata = MultiTableMetadata()

metadata.add_table(
    name='Customer',
    data=main_tables['Customer'],
    primary_key='customer_id'
)

metadata.add_table(
    name='Policy',
    data=main_tables['Policy'],
    primary_key='policy_id',
    parent='Customer',
    foreign_key='customer_id'
)

metadata.add_table(
    name='Claim',
    data=main_tables['Claim'],
    primary_key='claim_id',
    parent='Policy',
    foreign_key='policy_id'
)

metadata.add_table(
    name='Beneficiary',
    data=main_tables['Beneficiary'],
    primary_key='beneficiary_id',
    parent='Claim',
    foreign_key='claim_id'
)

metadata.add_table(
    name='Coverage',
    data=main_tables['Coverage'],
    primary_key='coverage_id',
    parent='Policy',
    foreign_key='policy_id'
)

metadata.add_table(
    name='Premium',
    data=main_tables['Premium'],
    primary_key='premium_id',
    parent='Policy',
    foreign_key='policy_id'
)

# Lookup Tables
metadata.add_table(
    name='PolicyStatus',
    data=lookup_tables['PolicyStatus'],
    primary_key='status_id'
)
metadata.add_table(
    name='PolicyType',
    data=lookup_tables['PolicyType'],
    primary_key='type_id'
)
metadata.add_table(
    name='CoverageType',
    data=lookup_tables['CoverageType'],
    primary_key='coverage_type_id'
)
metadata.add_table(
    name='PaymentMethod',
    data=lookup_tables['PaymentMethod'],
    primary_key='method_id'
)

# Define foreign keys to lookup tables
metadata.set_foreign_key('Policy', 'status_id', 'PolicyStatus')
metadata.set_foreign_key('Policy', 'type_id', 'PolicyType')
metadata.set_foreign_key('Coverage', 'coverage_type_id', 'CoverageType')
metadata.set_foreign_key('Premium', 'payment_method_id', 'PaymentMethod')

# ========== FIT MODEL ==========
print("Fitting HMA1 model on relational data...")
model =HMASynthesize(metadata)
model.fit(all_tables)
print("✅ Model training complete!")

# ========== SYNTHETIC ROW COUNTS ==========
sample_sizes = {
    'Customer': 100000,
    'Policy': 200000,
    'Claim': 350000,
    'Beneficiary': 300000,
    'Coverage': 350000,
    'Premium': 200000
}

# ========== GENERATE SYNTHETIC DATA ==========
synthetic_data = model.sample(scale=1.0, max_rows=sample_sizes)

# ========== SAVE SYNTHETIC OUTPUT ==========
os.makedirs("synthetic_output", exist_ok=True)
for table_name, df in synthetic_data.items():
    df.to_csv(f"synthetic_output/{table_name}.csv", index=False)
    print(f"✅ Saved: synthetic_output/{table_name}.csv")

print("\n🎉 All synthetic tables generated with referential integrity.")

In [ ]:
import pandas as pd
from sdv.multi_table import HMASynthesizer
from sdv.metadata.multi_table import MultiTableMetadata

# ======== Load Main Tables ========
main_tables = {
    'Customer': pd.read_csv('Customer.csv'),
    'Policy': pd.read_csv('Policy.csv'),
    'Claim': pd.read_csv('Claim.csv'),
    'Coverage': pd.read_csv('Coverage.csv'),
    'Premium': pd.read_csv('Premium.csv'),
    'Beneficiary': pd.read_csv('Beneficiary.csv')
}

# ======== Define Metadata ========

metadata = MultiTableMetadata()

# Add tables with just name and data
metadata.add_table('Customer', main_tables['Customer'])
metadata.add_table('Policy', main_tables['Policy'])
metadata.add_table('Claim', main_tables['Claim'])
metadata.add_table('Coverage', main_tables['Coverage'])
metadata.add_table('Premium', main_tables['Premium'])
metadata.add_table('Beneficiary', main_tables['Beneficiary'])

# Now manually define the metadata details for each table
metadata.tables['Customer']['primary_key'] = 'customer_id'

metadata.tables['Policy']['primary_key'] = 'policy_id'
metadata.tables['Policy']['fields'] = {
    'customer_id': {
        'ref': {
            'table': 'Customer',
            'field': 'customer_id'
        }
    }
}

metadata.tables['Claim']['primary_key'] = 'claim_id'
metadata.tables['Claim']['fields'] = {
    'policy_id': {
        'ref': {
            'table': 'Policy',
            'field': 'policy_id'
        }
    }
}

metadata.tables['Coverage']['primary_key'] = 'coverage_id'
metadata.tables['Coverage']['fields'] = {
    'policy_id': {
        'ref': {
            'table': 'Policy',
            'field': 'policy_id'
        }
    }
}

metadata.tables['Premium']['primary_key'] = 'premium_id'
metadata.tables['Premium']['fields'] = {
    'policy_id': {
        'ref': {
            'table': 'Policy',
            'field': 'policy_id'
        }
    }
}

metadata.tables['Beneficiary']['primary_key'] = 'beneficiary_id'
metadata.tables['Beneficiary']['fields'] = {
    'policy_id': {
        'ref': {
            'table': 'Policy',
            'field': 'policy_id'
        }
    }
}

# Validate metadata
metadata.validate(main_tables)



# ======== Fit the HMASynthesizer ========
synthesizer = HMASynthesizer(metadata)
synthesizer.fit(main_tables)


In [ ]:
# ======== Generate Oversized Sample ========
synthetic_data = synthesizer.sample(scale=10)  # scale > 1 to ensure sufficient rows

# ======== Trim to desired counts ========
desired_counts = {
    'Customer': 500,
    'Policy': 1200,
    'Claim': 2000,
    'Coverage': 1800,
    'Premium': 1600,
    'Beneficiary': 800
}

final_output = {}

for table_name, df in synthetic_data.items():
    count = desired_counts.get(table_name, len(df))
    final_output[table_name] = df.sample(n=min(count, len(df)), random_state=42).reset_index(drop=True)

# ======== Save final tables ========
for table, df in final_output.items():
    df.to_csv(f'synthetic_{table}.csv', index=False)


In [ ]:
# STEP 0: Install required library (run this cell once)
!pip install -q sdv ipywidgets

# STEP 1: Import libraries
import pandas as pd
import numpy as np
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata

In [ ]:


# Optional: For uploading files via widget in Jupyter (uncomment if you want)
# import ipywidgets as widgets
# from IPython.display import display
#
# upload_widget = widgets.FileUpload(accept='.csv', multiple=True)
# display(upload_widget)
#
# # After uploading files, you can access them via upload_widget.value
# # But manual loading from disk is simpler for multiple files

# STEP 2: Load CSV files manually from current folder
# Place your CSV files in the same directory as this notebook or specify the path

csv_files = [
    'Customer.csv',
    'Policy.csv',
    'Coverage.csv',
    'Premium.csv',
    'Claim.csv',
    'Beneficiary.csv',
    'PolicyStatus.csv',
    'PolicyType.csv',
    'CoverageType.csv',
    'PaymentMethod.csv'
]

dataframes = {}
for filename in csv_files:
    try:
        df_name = filename.replace('.csv', '')
        dataframes[df_name] = pd.read_csv(filename)
        print(f"Loaded {filename} with columns: {dataframes[df_name].columns.tolist()}")
    except FileNotFoundError:
        print(f"File {filename} not found. Please make sure it's in the notebook folder.")

# STEP 3: Define synthetic sample sizes
samples = {
    'Customer': 100,
    'Policy': 200,
    'Coverage': 200,
    'Premium': 150,
    'Claim': 400,
    'Beneficiary': 300
}

# STEP 4: Reusable CTGAN function
def generate_ctgan(data, table_name, num_rows):
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(data)
    model = CTGANSynthesizer(metadata)
    model.fit(data)
    synthetic = model.sample(num_rows=num_rows)
    print(f"✅ Generated synthetic data for {table_name}")
    return synthetic

# STEP 5: Generate synthetic data
synthetic_data = {}

# Customer (independent)
synthetic_data['Customer'] = generate_ctgan(dataframes['Customer'], 'Customer', samples['Customer'])

# Policy
policy_input = dataframes['Policy'].copy()
policy_input['customer_id'] = np.random.choice(synthetic_data['Customer']['customer_id'], size=len(policy_input))
policy_input['status_id'] = np.random.choice(dataframes['PolicyStatus']['status_id'], size=len(policy_input))
policy_input['type_id'] = np.random.choice(dataframes['PolicyType']['type_id'], size=len(policy_input))
synthetic_data['Policy'] = generate_ctgan(policy_input, 'Policy', samples['Policy'])

# Coverage
coverage_input = dataframes['Coverage'].copy()
coverage_input['policy_id'] = np.random.choice(synthetic_data['Policy']['policy_id'], size=len(coverage_input))
coverage_input['coverage_type_id'] = np.random.choice(dataframes['CoverageType']['coverage_id'], size=len(coverage_input))
synthetic_data['Coverage'] = generate_ctgan(coverage_input, 'Coverage', samples['Coverage'])

# Premium
premium_input = dataframes['Premium'].copy()
premium_input['policy_id'] = np.random.choice(synthetic_data['Policy']['policy_id'], size=len(premium_input))
premium_input['payment_method_id'] = np.random.choice(dataframes['PaymentMethod']['method_id'], size=len(premium_input))
synthetic_data['Premium'] = generate_ctgan(premium_input, 'Premium', samples['Premium'])

# Claim
claim_input = dataframes['Claim'].copy()
claim_input['policy_id'] = np.random.choice(synthetic_data['Policy']['policy_id'], size=len(claim_input))
synthetic_data['Claim'] = generate_ctgan(claim_input, 'Claim', samples['Claim'])

# Beneficiary
beneficiary_input = dataframes['Beneficiary'].copy()
beneficiary_input['policy_id'] = np.random.choice(synthetic_data['Policy']['policy_id'], size=len(beneficiary_input))
synthetic_data['Beneficiary'] = generate_ctgan(beneficiary_input, 'Beneficiary', samples['Beneficiary'])

# STEP 6: Save synthetic CSV files locally
for table_name, df in synthetic_data.items():
    filename = f"{table_name}_synthetic.csv"
    df.to_csv(filename, index=False)
    print(f"Saved synthetic data to {filename}")

# Now you can download these CSVs from the Jupyter file browser or open them directly.


In [ ]:
import pandas as pd
import numpy as np
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata
import joblib  # for saving models

# Load your dataframes here as before (omitted for brevity)

# Define a function to train and save a model
def train_and_save_ctgan(data, table_name, model_path):
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(data)
    model = CTGANSynthesizer(metadata)
    print(f"Training model for {table_name} on {len(data)} rows...")
    model.fit(data)
    joblib.dump(model, model_path)
    print(f"Saved model for {table_name} at {model_path}")

# Train & save models for all tables (example for Customer and Policy)
train_and_save_ctgan(dataframes['Customer'], 'Customer', 'Customer_ctgan.pkl')

# For dependent tables, adjust foreign keys before training if needed
policy_input = dataframes['Policy'].copy()
policy_input['customer_id'] = np.random.choice(dataframes['Customer']['customer_id'], size=len(policy_input))
train_and_save_ctgan(policy_input, 'Policy', 'Policy_ctgan.pkl')

# Repeat for Coverage, Premium, Claim, Beneficiary similarly


In [1]:
import pandas as pd
import numpy as np
import os
import joblib
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata
csv_files = [
    'Customer.csv',
    'Policy.csv',
    'PolicyStatus.csv',
    'PolicyType.csv',
    'Coverage.csv',
    'CoverageType.csv',
    'Premium.csv',
    'PaymentMethod.csv',
    'Claim.csv',
    'Beneficiary.csv'
]

dataframes = {}

for file in csv_files:
    if os.path.exists(file):
        df_name = file.replace('.csv', '')
        dataframes[df_name] = pd.read_csv(file)
        print(f"✅ Loaded {file} into dataframes['{df_name}'] with shape {dataframes[df_name].shape}")
    else:
        print(f"❌ File not found: {file}")

# Now you can verify all keys loaded:
print("\nAll loaded tables:", list(dataframes.keys()))

✅ Loaded Customer.csv into dataframes['Customer'] with shape (5000, 6)
✅ Loaded Policy.csv into dataframes['Policy'] with shape (12491, 7)
✅ Loaded PolicyStatus.csv into dataframes['PolicyStatus'] with shape (3, 2)
✅ Loaded PolicyType.csv into dataframes['PolicyType'] with shape (3, 2)
✅ Loaded Coverage.csv into dataframes['Coverage'] with shape (31398, 4)
✅ Loaded CoverageType.csv into dataframes['CoverageType'] with shape (3, 2)
✅ Loaded Premium.csv into dataframes['Premium'] with shape (12517, 5)
✅ Loaded PaymentMethod.csv into dataframes['PaymentMethod'] with shape (3, 2)
✅ Loaded Claim.csv into dataframes['Claim'] with shape (24993, 5)
✅ Loaded Beneficiary.csv into dataframes['Beneficiary'] with shape (18750, 6)

All loaded tables: ['Customer', 'Policy', 'PolicyStatus', 'PolicyType', 'Coverage', 'CoverageType', 'Premium', 'PaymentMethod', 'Claim', 'Beneficiary']


In [2]:


def train_and_save_ctgan(data, table_name, model_path, epochs=200):
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(data)

    model = CTGANSynthesizer(metadata, epochs=epochs, batch_size=256, pac=1)  # pac=1 to avoid this error
    print(f"Starting training {table_name} model with {len(data)} rows and {epochs} epochs...")
    model.fit(data)
    print(f"{table_name} model training complete.")

    joblib.dump(model, model_path)
    print(f"Saved {table_name} model to '{model_path}'")


In [3]:
train_and_save_ctgan(dataframes['Customer'], 'Customer', 'Customer_ctgan.pkl', epochs=200)

C:\Users\2000147874\Downloads\myenv\Lib\site-packages\sdv\single_table\base.py:162: FutureWarning:

The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.

C:\Users\2000147874\Downloads\myenv\Lib\site-packages\sdv\single_table\base.py:128: UserWarning:

We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.



Starting training Customer model with 5000 rows and 200 epochs...
PerformanceAlert: Using the CTGANSynthesizer on this data is not recommended. To model this data, CTGAN will generate a large number of columns.

Original Column Name   Est # of Columns (CTGAN)
date_of_birth          11
address                5000

We recommend preprocessing discrete columns that can have many values, using 'update_transformers'. Or you may drop columns that are not necessary to model. (Exit this script using ctrl-C)


KeyboardInterrupt: 

In [ ]:
policy_data = dataframes['Policy'].copy()
policy_data['customer_id'] = np.random.choice(dataframes['Customer']['customer_id'], size=len(policy_data))
policy_data['status_id'] = np.random.choice(dataframes['PolicyStatus']['status_id'], size=len(policy_data))
policy_data['type_id'] = np.random.choice(dataframes['PolicyType']['type_id'], size=len(policy_data))

train_and_save_ctgan(dataframes['Policy'], 'Policy', 'Policy_ctgan.pkl', epochs=50)


In [ ]:
coverage_data = dataframes['Coverage'].copy()
coverage_data['policy_id'] = np.random.choice(dataframes['Policy']['policy_id'], size=len(coverage_data))
coverage_data['coverage_type_id'] = np.random.choice(dataframes['CoverageType']['coverage_type_id'], size=len(coverage_data))

train_and_save_ctgan(coverage_data, 'Coverage', 'Coverage_ctgan.pkl', epochs=50)


In [ ]:
premium_data = dataframes['Premium'].copy()
premium_data['policy_id'] = np.random.choice(dataframes['Policy']['policy_id'], size=len(premium_data))
premium_data['payment_method_id'] = np.random.choice(dataframes['PaymentMethod']['method_id'], size=len(premium_data))

train_and_save_ctgan(premium_data, 'Premium', 'Premium_ctgan.pkl', epochs=50)


In [ ]:
claim_data = dataframes['Claim'].copy()
claim_data['policy_id'] = np.random.choice(dataframes['Policy']['policy_id'], size=len(claim_data))

train_and_save_ctgan(claim_data, 'Claim', 'Claim_ctgan.pkl', epochs=50)


In [ ]:
beneficiary_data = dataframes['Beneficiary'].copy()
beneficiary_data['policy_id'] = np.random.choice(dataframes['Policy']['policy_id'], size=len(beneficiary_data))

train_and_save_ctgan(beneficiary_data, 'Beneficiary', 'Beneficiary_ctgan.pkl', epochs=50)


In [ ]:
# ===== STEP 0: Install Dependencies (if not already installed) =====
# !pip install -q sdv joblib

# ===== STEP 1: Import Libraries =====
import pandas as pd
import numpy as np
import joblib
import os

# ===== STEP 2: Load Original Lookup Data (for FKs) =====
lookup_tables = {
    'Customer': pd.read_csv('Customer.csv'),
    'Policy': pd.read_csv('Policy.csv'),
    'PolicyStatus': pd.read_csv('PolicyStatus.csv'),
    'PolicyType': pd.read_csv('PolicyType.csv'),
    'Coverage': pd.read_csv('Coverage.csv'),
    'CoverageType': pd.read_csv('CoverageType.csv'),
    'Premium': pd.read_csv('Premium.csv'),
    'PaymentMethod': pd.read_csv('PaymentMethod.csv'),
    'Claim': pd.read_csv('Claim.csv'),
    'Beneficiary': pd.read_csv('Beneficiary.csv'),
}

# ===== STEP 3: Define how many rows to generate per table =====
sample_counts = {
    'Customer': 100000,      
    'Policy': 200000,
    'Coverage': 300000,
    'Premium': 250000,
    'Claim': 400000,
    'Beneficiary': 250000
}

# ===== STEP 4: Load Models and Generate Data =====
def load_model_and_sample(model_path, num_rows):
    model = joblib.load(model_path)
    synthetic = model.sample(num_rows=num_rows)
    print(f"✅ Generated {num_rows} synthetic rows from model {model_path}")
    return synthetic

# ===== STEP 5: Generate synthetic tables =====
synthetic_tables = {}

# 1. Customer (no FK)
synthetic_tables['Customer'] = load_model_and_sample('Customer_ctgan.pkl', sample_counts['Customer'])

# 2. Policy (FKs: customer_id, status_id, type_id)
policy = load_model_and_sample('Policy_ctgan.pkl', sample_counts['Policy'])
policy['customer_id'] = np.random.choice(synthetic_tables['Customer']['customer_id'], size=len(policy))
policy['status_id'] = np.random.choice(lookup_tables['PolicyStatus']['status_id'], size=len(policy))
policy['type_id'] = np.random.choice(lookup_tables['PolicyType']['type_id'], size=len(policy))
synthetic_tables['Policy'] = policy

# 3. Coverage (FKs: policy_id, coverage_type_id)
coverage = load_model_and_sample('Coverage_ctgan.pkl', sample_counts['Coverage'])
coverage['policy_id'] = np.random.choice(synthetic_tables['Policy']['policy_id'], size=len(coverage))
coverage['coverage_type_id'] = np.random.choice(lookup_tables['CoverageType']['coverage_type_id'], size=len(coverage))
synthetic_tables['Coverage'] = coverage

# 4. Premium (FKs: policy_id, payment_method_id)
premium = load_model_and_sample('Premium_ctgan.pkl', sample_counts['Premium'])
premium['policy_id'] = np.random.choice(synthetic_tables['Policy']['policy_id'], size=len(premium))
premium['payment_method_id'] = np.random.choice(lookup_tables['PaymentMethod']['method_id'], size=len(premium))
synthetic_tables['Premium'] = premium

# 5. Claim (FK: policy_id)
claim = load_model_and_sample('Claim_ctgan.pkl', sample_counts['Claim'])
claim['policy_id'] = np.random.choice(synthetic_tables['Policy']['policy_id'], size=len(claim))
synthetic_tables['Claim'] = claim

# 6. Beneficiary (FK: policy_id)
beneficiary = load_model_and_sample('Beneficiary_ctgan.pkl', sample_counts['Beneficiary'])
beneficiary['policy_id'] = np.random.choice(synthetic_tables['Policy']['policy_id'], size=len(beneficiary))
synthetic_tables['Beneficiary'] = beneficiary

# ===== STEP 6: Save to CSV =====
output_folder = 'synthetic_output'
os.makedirs(output_folder, exist_ok=True)

for table_name, df in synthetic_tables.items():
    file_path = os.path.join(output_folder, f"{table_name}_synthetic.csv")
    df.to_csv(file_path, index=False)
    print(f"📁 Saved {table_name} synthetic data to {file_path}")


In [ ]:
# ======= STEP 4: Load models and generate synthetic data =======
def load_and_generate_ctgan(model_path, num_samples):
    model = joblib.load(model_path)
    synthetic_data = model.sample(num_samples)
    print(f"Generated {num_samples} rows from model '{model_path}'")
    return synthetic_data

# Define how many samples you want per table
samples = {
    'Customer': 100,
    'Policy': 200,
    'Coverage': 200,
    'Premium': 150,
    'Claim': 400,
    'Beneficiary': 300
}

# Generate synthetic Customer data (independent)
synthetic_customer = load_and_generate_ctgan('Customer_ctgan.pkl', samples['Customer'])

# Generate synthetic Policy data — use synthetic_customer IDs for FK consistency
policy_data = dataframes['Policy'].copy()
policy_data['customer_id'] = np.random.choice(synthetic_customer['customer_id'], size=len(policy_data))
policy_data['status_id'] = np.random.choice(dataframes['PolicyStatus']['status_id'], size=len(policy_data))
policy_data['type_id'] = np.random.choice(dataframes['PolicyType']['type_id'], size=len(policy_data))
synthetic_policy = load_and_generate_ctgan('Policy_ctgan.pkl', samples['Policy'])

# Generate synthetic Coverage data — use synthetic_policy IDs
coverage_data = dataframes['Coverage'].copy()
coverage_data['policy_id'] = np.random.choice(synthetic_policy['policy_id'], size=len(coverage_data))
coverage_data['coverage_type_id'] = np.random.choice(dataframes['CoverageType']['coverage_type_id'], size=len(coverage_data))
synthetic_coverage = load_and_generate_ctgan('Coverage_ctgan.pkl', samples['Coverage'])

# Generate synthetic Premium data — use synthetic_policy IDs
premium_data = dataframes['Premium'].copy()
premium_data['policy_id'] = np.random.choice(synthetic_policy['policy_id'], size=len(premium_data))
premium_data['payment_method_id'] = np.random.choice(dataframes['PaymentMethod']['payment_method_id'], size=len(premium_data))
synthetic_premium = load_and_generate_ctgan('Premium_ctgan.pkl', samples['Premium'])

# Generate synthetic Claim data — use synthetic_policy IDs
claim_data = dataframes['Claim'].copy()
claim_data['policy_id'] = np.random.choice(synthetic_policy['policy_id'], size=len(claim_data))
synthetic_claim = load_and_generate_ctgan('Claim_ctgan.pkl', samples['Claim'])
# Generate synthetic Beneficiary data — use synthetic_policy IDs
beneficiary_data = dataframes['Beneficiary'].copy()
beneficiary_data['policy_id'] = np.random.choice(synthetic_policy['policy_id'], size=len(beneficiary_data))
synthetic_beneficiary = load_and_generate_ctgan('Beneficiary_ctgan.pkl', samples['Beneficiary'])

# Optional: Export synthetic data to CSV files
synthetic_customer.to_csv('synthetic_customer.csv', index=False)
synthetic_policy.to_csv('synthetic_policy.csv', index=False)
synthetic_coverage.to_csv('synthetic_coverage.csv', index=False)
synthetic_premium.to_csv('synthetic_premium.csv', index=False)
synthetic_claim.to_csv('synthetic_claim.csv', index=False)
synthetic_beneficiary.to_csv('synthetic_beneficiary.csv', index=False)

print("Synthetic data generated and saved to CSV files.")

In [ ]:
import pandas as pd
from sdv.metadata import Metadata
from sdv.lite import SingleTablePreset
from sdv.single_table import CTGANSynthesizer
import joblib


In [ ]:
# Load your data
df = pd.read_csv("Customer.csv")

# Clean column names (remove extra spaces, BOMs)
df.columns = df.columns.str.strip().str.replace('\ufeff', '')

# Drop rows with any nulls
df = df.dropna()

# Convert customer_id to integer (if it's float-like but actually int)
if df['customer_id'].dtype == 'float':
    df['customer_id'] = df['customer_id'].astype('int')


In [ ]:
# Create metadata automatically from the DataFrame
metadata = Metadata()
metadata.detect_from_dataframe(data=df)
